In [1]:
import pandas as pd
import json

### read data and conf

In [2]:
json_path = '../data/summarization/text_files'
data_file_name = 'txt_dataset_summarizer.json'
conf_file_name = 'txt_dataset_summarizer_config.json'

In [3]:
data = pd.read_json(f'{json_path}/{data_file_name}', orient='index')
with open(f'{json_path}/{conf_file_name}') as f:
    conf_json = json.load(f)

data.head(3)

,text,word_count,golden,notion,grammarly,current_doc_summarizier,current_update_summarizer,version_1_summarizer,version_2_summarizer,version_3_summarizer,version_2_turbo_summarizer
0,(CNN)A mammoth fire broke out Friday morning i...,143,Fire breaks out at the General Electric Applia...,A massive fire broke out at the General Electr...,A massive fire broke out at the General Electr...,A large fire broke out Friday morning in the G...,A large fire broke out Friday morning in the G...,Title: Fire Breaks Out in Kentucky Industrial ...,Title: Fire Breaks Out in Kentucky Industrial ...,A fire broke out Friday morning in a Kentucky ...,Title: Mammoth fire breaks out at Kentucky ind...
1,"Hull, Leicester and Swansea City are following...",147,Hull's chief scout Stan Ternent has watched M...,"Hull, Leicester, and Swansea City are interest...","Hull, Leicester, and Swansea City are interest...","Maciej Rybus, a 25-year-old Polish internation...","Maciej Rybus, a 25-year-old Polish internation...","Title: Maciej Rybus Followed by Hull, Leiceste...","Title: Maciej Rybus Followed by Hull, Leiceste...","Hull, Leicester and Swansea City are intereste...","Title: Hull, Leicester and Swansea interested ..."
2,Sportsmail have teamed up with Golfbidder to o...,123,Sportsmail have teamed up with Golfbidder for ...,Sportsmail and Golfbidder have partnered to of...,Sportsmail and Golfbidder are running a compet...,Sportsmail have partnered with Golfbidder to o...,Sportsmail and Golfbidder have teamed up to of...,Title: Win a Callaway Golf Prize Bundle Worth ...,Title: Win a Callaway Golf Prize Bundle Worth ...,One lucky reader has the chance to win a bundl...,Title: Win a bundle of Callaway golf clubs and...


In [7]:
print(data.iloc[5]['grammarly'])

Workforms has recently undergone updates, including support for the People Column and a 1:1 question format. The Connect Board Column and a submission analytics page are also coming soon. Check out the details on their website.


# Building new prompts

In [6]:
import openai
import os
from dotenv import load_dotenv
from modules.openai.OpenaiModels import OpenaiCompletionWordLimit, OpenaiChatCompletionWordLimit

load_dotenv()

azure_openai_key = os.environ.get('AZURE_OPENAI_KEY')
azure_openai_endpoint = os.environ.get('AZURE_OPENAI_ENDPOINT')

openai.api_key = azure_openai_key
openai.api_base = azure_openai_endpoint
openai.api_type = "azure"

In [33]:
system_prompt = """You are a helpful assistant. Using your deep linguistic understanding and knowledge, condense the essence of the content provided by the user.\n Retain the core ideas, crucial details, and significant context while omitting any extraneous information.\n Please deliver a concise and coherent summary of the provided text.\n Respond only in string using less than {max_words} words.\n The output MUST be in the following structure:\nTitle: <the title>\n<the summary>."""

user_prompt = """Please summarize the following text:{input_text}"""



version_3_5 = OpenaiChatCompletionWordLimit(model_name="gpt-3-5-turbo",
                                            api_version="2023-05-15",
                                            temperature=0.0,
                                            max_tokens=200,
                                            system_prompt_template=system_prompt,
                                            user_prompt_template=user_prompt,
                                            max_word_ratio=0.75)


In [1]:
# version_2_template = """Using your deep linguistic understanding and knowledge, condense the essence of the provided content.\n Retain the core ideas, crucial details, and significant context while omitting any extraneous information.\n Please deliver a concise and coherent summary of the following text.\n The summary MUST be shorter or equal to {max_words} words.\n The output MUST be a JSON object with the following schema:
# {{"title":<the title>,"body":<the summary>}}\n----------------------\n\nThe text to summarize: {input_text}\n The output:\n"""
# 
# version_2_sum = OpenaiCompletionWordLimit(
#     model_name="text-davinci-003",
#     api_version="2023-05-15",
#     temperature=0.0,
#     max_tokens=512,
#     prompt_template=version_2_template,
#     max_word_ratio=0.75
# )

### Apply on dataset

In [16]:
model_name = "version_2_turbo_summarizer"

In [17]:
# completion api
data[model_name] = data['text'].apply(version_3_5.call_llm)

In [18]:
print(data.iloc[0].version_2_summarizer)

Title: Fire Breaks Out in Kentucky Industrial Park
A fire broke out Friday morning in the General Electric Appliance Park in Louisville, Kentucky. No injuries or trapped people were reported. Video showed smoke and orange flames. Firefighters sprayed water from the periphery. The cause of the fire is unknown. The park is large, with one warehouse being able to fit 34 football fields.


In [24]:
print(data.iloc[5][model_name])

Title: Updates to Workforms

Workforms has made updates to their platform, including support for the People Column and a 1:1 question format. Coming soon are the Connect Board Column and a submission analytics page for reporting on submission rate, average submission time, and form submitter geolocation, browser, and device. More details can be found on their website.


In [3]:
print("\nWe\u2019ve made a few updates to Workforms recently, so we thought we\u2019d summarise the enhancements into a  short and sweet  post for our community!\nNEW UPDATES\n\u2022 The People Column is now supported: Assign an item to a person or team without creating an automation!\n\u2022 1:1 question format\nCOMING SOON \n\u2022 The Connect Board Column: Specify which items of the connected board are related to his\/her submission.\n\u2022 Submission analytics page: Take reporting to the next level, pull insightful analytics such as submission rate, average submission time, and split by geolocation, browser, and device of the form submitters.\nCheck out more details here 24 and stay tuned! \n")


We’ve made a few updates to Workforms recently, so we thought we’d summarise the enhancements into a  short and sweet  post for our community!
NEW UPDATES
• The People Column is now supported: Assign an item to a person or team without creating an automation!
• 1:1 question format
COMING SOON 
• The Connect Board Column: Specify which items of the connected board are related to his\/her submission.
• Submission analytics page: Take reporting to the next level, pull insightful analytics such as submission rate, average submission time, and split by geolocation, browser, and device of the form submitters.
Check out more details here 24 and stay tuned! 


In [73]:
# data = data.drop('larium_devinci_summarizer',axis=1)

### update config

In [21]:
conf_json.update({model_name: version_3_5.get_config()})

In [22]:
conf_json

{'current_doc_summarizer': {'model_name': 'text-davinci-003',
  'api_version': '2023-05-15',
  'temperature': 0.0,
  'max_tokens': 200,
  'prompt_template': 'summarize the following text: {input_text}'},
 'current_update_summarizer': {'model_name': 'text-davinci-003',
  'api_version': '2023-05-15',
  'temperature': 0.0,
  'max_tokens': 200,
  'prompt_template': 'summarize the following text: {input_text}'},
 'version_1_summarizer': {'model_name': 'text-davinci-003',
  'api_version': '2023-05-15',
  'temperature': 0.0,
  'max_tokens': 200,
  'prompt_template': 'Summarize the key points of the text provided between the []. The summary MUST be concise and shorter than {max_words} words. Also, the output should be in the following structure: \n\nTitle: <the title>\n\n<the summary>.  \n----------------------\nThe text to summarize: [{input_text}]\n\nThe concise summary:\n',
  'max_word_ratio': 0.6},
 'version_2_summarizer': {'model_name': 'text-davinci-003',
  'api_version': '2023-05-15',
 

## Save data file and conf json

In [23]:
# save
data.to_json(f"{json_path}/{data_file_name}", orient='index', indent=4, force_ascii=True)
with open(f'{json_path}/{conf_file_name}', 'w', encoding='utf-8') as f:
    json.dump(conf_json, f, ensure_ascii=False, indent=4)